#### PCA + Standardizer + sklearn/SVM with MNIST

##### Libraries

In [ ]:
using MLDatasets           # mnist
using Images
using PreprocessingImages; pim = PreprocessingImages
using PreprocessingArrays; pa  = PreprocessingArrays

using MLJ                  # make_blobs, rmse, confmat, categorical
using MLDataUtils          # label, nlabel, labelfreq
using PyCall               # required by ScikitLearn

using Metrics              # r2-score
using Random
using Plots; gr()
using StatsPlots
using DataFrames

##### Functions

In [ ]:
# metrics
function printMetrics(ŷ, y)
    display(confmat(ŷ, y))
    println("accuracy: ", round(accuracy(ŷ, y); digits=3))
    println("f1-score: ", round(multiclass_f1score(ŷ, y); digits=3))
end

##### MNIST

In [ ]:
# load mnist
datasetX,    datasetY    = MNIST(:train)[:]
validationX, validationY = MNIST(:test)[:]

display( size(datasetX) )

img  = datasetX[:, :, 1:5]
img2 = permutedims(img, (2, 1, 3))

display(datasetY[1:5]')
mosaicview( Gray.(img2)  ; nrow=1)

In [ ]:
# split trainset, testset from dataset
Random.seed!(1)
(trainX, trainY), (testX, testY) = stratifiedobs((datasetX, datasetY), p = 0.7)
size(trainX), size(testX), size(validationX)

##### Preprocessing

In [ ]:
function preprocess(X, y)
    newX = pim.batchImage2DF(X)
    #coerce!(newX)   # no need, all scitypes are Continuous in this example
    new_y = coerce(y, OrderedFactor)
    
    return (newX, new_y)
end

X_tr, y_tr = preprocess(trainX, trainY);

In [ ]:
scitype(X_tr)

In [ ]:
scitype(y_tr)

##### Pipe the model

In [ ]:
# reduce predictors
PCA = @load PCA pkg=MultivariateStats verbosity=0
reducer = PCA(pratio = 0.9)

# standardize predictors
std = Standardizer()

# svm scikitlearn
SVMClass = @load SVMClassifier pkg="ScikitLearn" verbosity=0
svm = SVMClass()

pipe = @pipeline reducer std svm

##### Training

In [ ]:
mach = MLJ.machine(pipe, X_tr, y_tr) |> fit!

##### Testing

In [ ]:
X_ts, y_ts = preprocess(testX, testY)
ŷ = MLJ.predict(mach, X_ts)
printMetrics(ŷ, y_ts)

##### Validation

In [ ]:
X_v, y_v = preprocess(validationX, validationY)
ŷ = MLJ.predict(mach, X_v)
printMetrics(ŷ, y_v)